In [ ]:
!git clone https://github.com/jaimin-k/Usim


In [ ]:
!ls Usim


In [ ]:
!pip install keras
!pip3 install imgaug

!pip3 install imgaug

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical 

from keras.layers import Dropout,Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from imgaug import augmenters as iaa
import random
import pickle
import cv2
import pandas as pd
import ntpath
import os

Reading the data

In [ ]:
datadir = 'Usim'
columns = ['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed']
data = pd.read_csv(os.path.join(datadir, 'driving_log.csv'), names = columns)
pd.set_option('display.max_colwidth', -1)
data.head()
def path_leaf(path):
  head, tail = ntpath.split(path)
  return tail
data['center'] = data['center'].apply(path_leaf)
data['left'] = data['left'].apply(path_leaf)
data['right'] = data['right'].apply(path_leaf)

In [ ]:
def path_leaf(path):
  head,tail = ntpath.split(path)
  return tail
data['center']=data['center'].apply(path_leaf)
data['left']=data['left'].apply(path_leaf)
data['right']=data['right'].apply(path_leaf)
#data.head()


Construct histogram


In [ ]:
num_bins = 26
samples_per_bin = 690
hist, bins = np.histogram(data['steering'], num_bins)
center = (bins[:-1]+ bins[1:]) * 0.5
plt.bar(center, hist, width=0.05)
plt.plot((np.min(data['steering']), np.max(data['steering'])), (samples_per_bin, samples_per_bin))

flatten extraneous samples for specific bins whose freq. exceed 200 #preprocessing
making distribution more uniform further

In [ ]:
print("total data",len(data))
remove_list =[]
#looping through samples of each bin
for j in range(num_bins):
  list_=[]
  for i in range(len(data['steering'])):
    if data['steering'][i]>= bins[j] and data['steering'][i] <= bins[j+1]: #if ssteering angle falls between two bins eg -0.72,-0.69 will all belong to same bin
    #append each index that fall in specifc category in the list
      list_.append(i)
  list_ = shuffle(list_)
  list_ = list_[samples_per_bin:]
  remove_list.extend(list_)

print('removed',len(remove_list))
data.drop(data.index[remove_list],inplace=True)
print('remaining',len(data))

hist,_ = np.histogram(data['steering'],(num_bins))
plt.bar(center,hist, width=0.05)
plt.plot((np.min(data['steering']),np.max(data['steering'])),(samples_per_bin,samples_per_bin))

traing data corresponding steering angles as labels

In [ ]:
print(data.iloc[1])
def load_img_steering(datadir,df):
  image_path=[]
  steering=[]
  for i in range(len(data)):
    indexed_data = data.iloc[i] #iloc allows to perform selection of data from dataframe based on specified angle, index data will contain specific row of data of each iteration
    center,left,right = indexed_data[0], indexed_data[1],indexed_data[2]
    image_path.append(os.path.join(datadir, center.strip())) #gives specific path for each image
    steering.append(float(indexed_data[3]))
  image_paths = np.asarray(image_path)
  steerings = np.asarray(steering)
  return image_path, steerings
image_paths,steerings =  load_img_steering(datadir + '/IMG', data) #2 arrays-one for images and other with each images corresponding label-steering angle

**Split data collected from simulation into training and validation data**

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(image_paths, steerings, test_size= 0.2 , random_state=0)
print(f'Training Samples={len(X_train)}, Validation Samples= {len(X_val)}')

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(12,4))
axes[0].hist(y_train, bins=num_bins, width = 0.05 , color = 'blue')
axes[0].set_title('Training set')
axes[1].hist(y_val, bins=num_bins, width = 0.05 , color = 'red')
axes[1].set_title('Validation set')

In [ ]:
#Data augmentation-zooming-to get a closer look at some features in image
def zoom(image):
  zoom = iaa.Affine(scale=(1,1.3))
  image = zoom.augment_image(image)
  return image


In [ ]:
image = image_paths[random.randint(0,1000)]
original_image = mpimg.imread(image)
zoomed_image = zoom(original_image)
fig, axs = plt.subplots(1,2,figsize=(15,10))
fig.tight_layout()
axs[0].imshow(original_image)
axs[0].set_title('Original image')

axs[1].imshow(zoomed_image)
axs[1].set_title('zoomed image')

In [ ]:
def pan(image):
  pan = iaa.Affine(translate_percent={'x':(-0.1,0.1),'y':(-0.1,0.1)})#appine decides random percentage of pan
  image = pan.augment_image(image)
  return image

In [ ]:

#image = image_paths[random.randint(0,1000)]
#original_image = mpimg.imread(image)
#panned_image = pan(original_image)
#fig, axs = plt.subplots(1,2,figsize=(15,10))
#fig.tight_layout()
#axs[0].imshow(original_image)
#axs[0].set_title('Original image')

#axs[1].imshow(panned_image)
#axs[1].set_title('panned image')

image = image_paths[random.randint(0,1000)]
original_image = mpimg.imread(image)
panned_image = pan(original_image)
fig, axs = plt.subplots(1,2,figsize=(15,10))
fig.tight_layout()
axs[0].imshow(original_image)
axs[0].set_title('Original image')

axs[1].imshow(panned_image)
axs[1].set_title('panned image')

In [ ]:
def img_random_brightness(image):
  brightness = iaa.Multiply((0.2,1.2))
  image = brightness.augment_image(image)
  return image

In [ ]:
image = image_paths[random.randint(0,1000)]
original_image = mpimg.imread(image)
Brighteness_altered_image = img_random_brightness(original_image)
fig, axs = plt.subplots(1,2,figsize=(15,10))
fig.tight_layout()
axs[0].imshow(original_image)
axs[0].set_title('Original image')

axs[1].imshow(Brighteness_altered_image)
axs[1].set_title('Brighteness altered image')

In [ ]:
def img_random_flip(image, steering_angle):
  image = cv2.flip(image,1 )#0-vertical slip 1:horizontal flip -1:hor,vert flip, 
  #also need to flip sterring angle
  steering_angle = -steering_angle
  return image, steering_angle

In [ ]:
random_index = random.randint(0,1000)
image = image_paths[random_index]
steering_angle = steerings[random_index]
original_image = mpimg.imread(image)
flipped_image, flipped_steering_angle = img_random_flip(original_image, steering_angle)
fig, axs = plt.subplots(1,2,figsize=(15,10))
fig.tight_layout()
axs[0].imshow(original_image)
axs[0].set_title('Original image-'+'Steering angle:' +str(steering_angle))

axs[1].imshow(flipped_image)
axs[1].set_title('flipped_image-'+'Steering angle:' +str(flipped_steering_angle))

randomizing occurance of augmentation functions

In [ ]:
def random_augment(image, steering_angle):
  image= mpimg.imread(image)
  if np.random.rand()<0.5:   #random func returns value between 0 - 1 with 0.5 ie equal probaility we are effectively running this if state. 50 % times
    image = pan(image) #pan augmentation applied only 50% of time
  if np.random.rand()<0.5:
    image = zoom(image)
  if np.random.rand()<0.5:
    image = img_random_brightness(image)
  if np.random.rand()<0.5:
    image, steering_angle = img_random_flip(image,steering_angle)  
  return image,steering_angle

plotting fully augmented images with combined augmentation techniqyues

In [ ]:
ncol = 2
nrow = 10
fig, axs = plt.subplots(nrow,ncol,figsize=(15,50))
fig.tight_layout()
for i in range(10): #each iteration random image is used to augment
  rand_num= random.randint(0,len(image_paths)-1)
  random_image = image_paths[rand_num]
  random_steering = steerings[rand_num]

  original_image = mpimg.imread(random_image)
  augmented_image,steering = random_augment(random_image, random_steering)

  axs[i][0].imshow(original_image)
  axs[i][0].set_title('Original image')

  axs[i][1].imshow(augmented_image)
  axs[i][1].set_title('Augmented image')

Preprocessing image data


In [ ]:
def img_preprocessing(img):
  #read image path snd store actial image that it contains
  img[60:135,:,:] #cropping height of image roi
  img = cv2.cvtColor(img,cv2.COLOR_RGB2YUV)
  img = cv2.GaussianBlur(img, (3,3),0)
  img = cv2.resize(img, (200,66)) #not necessary but helps to maintain consistent to make image size small and also matches input image size used by NVIDIA model architecture
  img = img/255 #normalization (no visual impact on image)
  return img

In [ ]:
image = image_paths[100]
original_image = mpimg.imread(image)
preprocessed_image = img_preprocessing(original_image)

fig, axes = plt.subplots(1,2, figsize=(15,10))
fig.tight_layout()
axes[0].imshow(original_image)
axes[0].set_title('original image')
axes[1].imshow(preprocessed_image)
axes[1].set_title('preprocessed image')


In [ ]:
def batch_generator(image_paths, steering_ang, batch_size, istraining):
  
  while True:
    batch_img = []
    batch_steering = []
    
    for i in range(batch_size):
      random_index = random.randint(0, len(image_paths) - 1)
      
      if istraining:
        im, steering = random_augment(image_paths[random_index], steering_ang[random_index])
     
      else:
        im = mpimg.imread(image_paths[random_index])
        steering = steering_ang[random_index]
      
      im = img_preprocessing(im)
      batch_img.append(im)
      batch_steering.append(steering)
    yield (np.asarray(batch_img), np.asarray(batch_steering))

In [ ]:
X_train_gen,y_train_gen = next(batch_generator(X_train,y_train,1,1))
X_val_gen,y_val_gen = next(batch_generator(X_val,y_val,1,0))

fig, axes = plt.subplots(1,2, figsize=(15,10))
fig.tight_layout()
axes[0].imshow(X_train_gen[0])
axes[0].set_title('Training image')
axes[1].imshow(X_val_gen[0])
axes[1].set_title('Validation image')

#validating imgs that are not augmented

Running dataset through preprocessing:training dataset

In [ ]:
#X_train = np.array(list(map(img_preprocessing, X_train)))#returns in form of list need to store as an array
#X_val = np.array(list(map(img_preprocessing, X_val)))


#to verify preprocessing successfuully

In [ ]:
#plt.imshow(X_train[random.randint(0,len(X_train)-1)])
#plt.axis('off')
#print(X_train.shape,X_val.shape)

In [ ]:
def nvidia_model():    
    model= Sequential()
     
    model.add(Convolution2D(24, (5, 5), (2, 2), input_shape=(66, 200, 3), activation='elu'))
    model.add(Convolution2D(36, (5, 5), (2, 2), activation='elu'))
    
    model.add(Convolution2D(48, (5, 5), (2, 2), activation='elu'))
    #model.add(Dropout(0.1))
    model.add(Convolution2D(64, (3, 3), activation='elu'))
    #we replaced it with sigmoid  to avoid vanishing gradient problem in nnet, relu can sometime cause dead relu where nn dies and on returns zero ie if value fed in relu is less than zer o relu passes zero if gradient is zero backpropogation won't update weight and model becomes stagnant and alwas returns zero
    #we can replace relu with elu -has non zero gradient vlaue in negative region,it always remain capable of learning and capable of reducing erroe in model
    model.add(Convolution2D(64, (3, 3), activation='elu'))
    #model.add(Dropout(0.5))
  
    model.add(Flatten())
    
    model.add(Dense(100, activation = 'elu'))
    
  
    model.add(Dense(50, activation = 'elu'))
    
  
    model.add(Dense(10, activation = 'elu'))
    
  
    model.add(Dense(1)) # output the predicted steering angle 

    optimizer = Adam(lr=0.001)
    model.compile(loss='mse', optimizer=optimizer)
    return model

model = nvidia_model()    
print(model.summary())

In [ ]:
history = model.fit_generator(batch_generator(X_train,y_train,batch_size=100, istraining=1), 
                              steps_per_epoch=150, 
                              epochs = 20, 
                              validation_data=batch_generator(X_val,y_val,100,0),
                              validation_steps=200,
                              verbose=1,
                              shuffle=1)

in final report can show differnce in loss and accuracy comparing relu and elu

In [ ]:

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training','validation'])
plt.title('loss')
plt.xlabel('epoch')

In [ ]:
model.save('model.h5')  

In [ ]:
from google.colab import files
files.download('model.h5')